In [6]:
import numpy as np
import nibabel as nib

In [1]:

#raise error if data type doesn't match 4D array -> function expects preprocessed BOLD fMRI file 
import numpy as np

# Define the dimensions of the 4D array (shape: (x, y, z, t))
x_dim = 10  # Number of rows in each 3D slice
y_dim = 10  # Number of columns in each 3D slice
z_dim = 5   # Number of slices
t_dim = 20  # Number of time points

# Generate a random 4D array
random_4d_array = np.random.rand(x_dim, y_dim, z_dim, t_dim)


In [ ]:

# Load BOLD resting state data
bold_file = nib.load(f"{folder_bold}BOLDrestingCAP_pp_final.nii")
bold_sequence = bold_file.get_fdata()
r, c, d, t = bold_sequence.shape
averaged_bold_seq = np.zeros((t, brain_par))

# Load atlas registered to the BOLD file
atlas_file = nib.load(f"{folder_bold}atlas_reg.nii.gz")
atlas = atlas_file.get_fdata()

# Averaging Bold signal
for frame in range(t):
    bold_frame = bold_sequence[:, :, :, frame]
    for b in range(1, brain_par + 1):
        temp = np.zeros(2)
        for all_d in range(d):
            indices_x, indices_y = np.where(atlas[:, :, all_d] == b)
            for kk in range(len(indices_x)):
                temp = np.append(temp, bold_frame[indices_x[kk], indices_y[kk], all_d])
        temp = temp[temp != 0]
        averaged_bold_seq[frame, b - 1] = np.mean(temp)

# Remove mean from the already averaged bold sequences
mean_sig = np.mean(averaged_bold_seq, axis=1)
for b in range(brain_par):
    averaged_bold_seq[:, b] -= mean_sig

if use_deconv:
    TR = 1.4
    thr = 1
    event_lag_max = round(10 / TR)
    averaged_bold_seq, onset, hrf, event_lag, PARA = wgr_deconv_canonhrf_par(
        averaged_bold_seq, thr, event_lag_max, TR
    )
